# Machine Learning 7

In this assignment, students will be using the K-nearest neighbors algorithm to predict how many points NBA players scored in the 2013-2014 season.

A look at the data:

Before we dive into the algorithm, let’s take a look at our data. Each row in the data contains information on how a player performed in the 2013-2014 NBA season.

Download 'nba_2013.csv' file from this link:
https://www.dropbox.com/s/b3nv38jjo5dxcl6/nba_2013.csv?dl=0

Here are some selected columns from the data:<br />
player - name of the player<br />
pos - the position of the player<br />
g - number of games the player was in<br />
gs - number of games the player started<br />
pts - total points the player scored<br />

There are many more columns in the data, mostly containing information about average player game performance over the course of the season.

See this site for an explanation of the rest of them.

**Read dataset and identify which columns are present in dataset**

In [53]:
import pandas

with open("./nba_2013.csv", "r") as csvfile:
    nba = pandas.read_csv(csvfile)

nba.columns.values

array(['player', 'pos', 'age', 'bref_team_id', 'g', 'gs', 'mp', 'fg',
       'fga', 'fg.', 'x3p', 'x3pa', 'x3p.', 'x2p', 'x2pa', 'x2p.', 'efg.',
       'ft', 'fta', 'ft.', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk',
       'tov', 'pf', 'pts', 'season', 'season_end'], dtype=object)

## Euclidean distance

Before we can predict using KNN, we need to find some way to figure out which data rows are “closest” to the row we’re trying to predict on.

We can use the principle of euclidean distance to find the most similar NBA players to Lebron James.

In [54]:
# Select Lebron James from our dataset
selected_player = nba[nba["player"] == "LeBron James"].iloc[0]

# Choose only the numeric columns (we'll use these to compute euclidean distance)
distance_columns = ['age', 'g', 'gs', 'mp', 'fg', 'fga', 'fg.', 'x3p', 'x3pa', 'x3p.', 'x2p', 'x2pa', 'x2p.', 'efg.', 'ft', 'fta', 'ft.', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']

def euclidean_distance(row):
    inner_value = 0
    for k in distance_columns:
        inner_value += (row[k] - selected_player[k]) ** 2
    return math.sqrt(inner_value)

# Find the distance from each player in the dataset to lebron.
lebron_distance = nba.apply(euclidean_distance, axis=1)

## Normalizing columns

You may have noticed that horsepower in the cars example had a much larger impact on the final distance than racing_stripes did. This is because horsepower values are much larger in absolute terms, and thus dwarf the impact of racing_stripes values in the euclidean distance calculations.

This can be bad, because a variable having larger values doesn’t necessarily make it better at predicting what rows are similar.

A simple way to deal with this is to normalize all the columns to have a mean of 0, and a standard deviation of 1. This will ensure that no single column has a dominant impact on the euclidean distance calculations.

To set the mean to 0, we have to find the mean of a column, then subtract the mean from every value in the column. To set the standard deviation to 1, we divide every value in the column by the standard deviation.

In [55]:
# Select only the numeric columns from the NBA dataset
nba_numeric = nba[distance_columns]

# Normalize all of the numeric columns
nba_normalized = (nba_numeric - nba_numeric.mean()) / nba_numeric.std()

## Finding the nearest neighbor

We now know enough to find the nearest neighbor of a given row in the NBA dataset. We can use the distance.euclidean function from scipy.spatial, a much faster way to calculate euclidean distance.

In [57]:
from scipy.spatial import distance

# Fill in NA values in nba_normalized
nba_normalized.fillna(0, inplace=True)

# Find the normalized vector for lebron james.
lebron_normalized = nba_normalized[nba["player"] == "LeBron James"]

# Find the distance between lebron james and everyone else.
euclidean_distances = nba_normalized.apply(lambda row: distance.euclidean(row, lebron_normalized), axis=1)

# Create a new dataframe with distances.
distance_frame = pandas.DataFrame(data={"dist": euclidean_distances, "idx": euclidean_distances.index})
distance_frame.sort_values("dist", inplace=True)

# Find the most similar player to lebron (the lowest distance to lebron is lebron, the second smallest is the most similar non-lebron player)
second_smallest = distance_frame.iloc[1]["idx"]
most_similar_to_lebron = nba.loc[int(second_smallest)]["player"]

## Generating training and testing sets

Now that we know how to find the nearest neighbors, we can make predictions on a test set. We’ll try to predict how many points a player scored using the 5 closest neighbors. We’ll find neighbors by using all the numeric columns in the dataset to generate similarity scores.

First, we have to generate test and train sets. In order to do this, we’ll use random sampling. We’ll randomly shuffle the index of the nba dataframe, and then pick rows using the randomly shuffled values.

If we didn’t do this, we’d end up predicting and training on the same data set, which would overfit. We could do cross validation also, which would be slightly better, but slightly more complex.

In [58]:
import random
from numpy.random import permutation

# Randomly shuffle the index of nba.
random_indices = permutation(nba_normalized.index)

# Set a cutoff for how many items we want in the test set (in this case 1/3 of the items)
test_cutoff = math.floor(len(nba_normalized)/3)

# Generate the test set by taking the first 1/3 of the randomly shuffled indices.
test = nba_normalized.loc[random_indices[1:test_cutoff]]

# Generate the train set with the rest of the data.
train = nba_normalized.loc[random_indices[test_cutoff:]]

## Using sklearn for k nearest neighbors

Sklearn performs the normalization and distance finding automatically, and lets us specify how many neighbors we want to look at.

In [59]:
# The columns that we will be making predictions with.
x_columns = ['age', 'g', 'gs', 'mp', 'fg', 'fga', 'fg.', 'x3p', 'x3pa', 'x3p.', 'x2p', 'x2pa', 'x2p.', 'efg.', 'ft', 'fta', 'ft.', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf']

# The column that we want to predict.
y_column = ["pts"]

from sklearn.neighbors import KNeighborsRegressor

# Create the knn model.
# Look at the five closest neighbors.
knn = KNeighborsRegressor(n_neighbors=5)

# Fit the model on the training data.
knn.fit(train[x_columns], train[y_column])

# Make point predictions on the test set using the fit model.
predictions = knn.predict(test[x_columns])
predictions

array([[ 1.0964998 ],
       [-0.91913625],
       [-1.00374109],
       [-0.50631562],
       [-0.34390832],
       [ 1.78354216],
       [ 1.87154821],
       [-1.01139379],
       [-0.73292055],
       [-0.09604589],
       [ 0.51914613],
       [-0.89362725],
       [ 0.98681111],
       [ 0.92941586],
       [-0.31754903],
       [-1.00586684],
       [ 0.42561314],
       [-1.09599864],
       [-0.81965115],
       [-0.82900445],
       [-0.66277081],
       [-0.72314211],
       [-0.98843569],
       [ 0.229619  ],
       [-0.11050099],
       [-0.26440528],
       [ 1.74017687],
       [ 1.87792546],
       [-1.02754949],
       [ 0.88902661],
       [-0.77713615],
       [-0.97227999],
       [ 1.19045795],
       [-0.21636333],
       [-0.50461502],
       [ 1.82903321],
       [-0.10752494],
       [ 0.11695425],
       [-0.93444164],
       [-0.69678281],
       [-0.38727362],
       [ 0.46047543],
       [-0.07266264],
       [ 1.74570382],
       [-1.01564529],
       [-0

## Computing error

Now that we know our point predictions, we can compute the error involved with our predictions.

In [60]:
# Get the actual values for the test set.
actual = test[y_column]

# Compute the mean squared error of our predictions.
mse = (((predictions - actual) ** 2).sum()) / len(predictions)
mse

pts    0.0292
dtype: float64